# Personalización de Activation Functions, Initializers, Regularizers, and Constraints

En general el proceso es el mismo que el indicado en la sección de pérdidas. Se crea una función que toma los parámetros necesarios y dentro se ejecuta la lógica personalizada que se quiera, devolviendo un valor del tipo necesario. O es posible también generar una subclase pertinente

In [5]:
import tensorflow as tf

tf.random.set_seed(42)

def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [7]:
layer = tf.keras.layers.Dense(1, activation=my_softplus,
                              kernel_initializer=my_glorot_initializer,
                              kernel_regularizer=my_l1_regularizer,
                              kernel_constraint=my_positive_weights)

1. La función activación se aplicará al output de esta capa densa y el resultado será input de la siguiente capa
2. Los pesos de la capa van a ser inicializados usando el valor retornado por el inicializador
3. En cada paso del entrenamiento los pesos van a pasar por la regularización para computar la pérdida por regularización, que van a ser sumados al peso principal para obtener la pérdida final usada para entrenar
4. El último paso, la función constraint va a ser llamada después de cada paso de entrenamiento, y los pesos de la capa van a ser reemplazados por los pesos constrained

Si se desean guardar los hyperparametros de una función con el modelo es necesario implementar una subclase de la entidad necesaria como
- tf.keras.regularizers.Regularizer
- tf.keras.constraints.Constraint
- tf.keras.optimizers.Optimizer
- tf.keras.initializers.Initializer
- tf.keras.layers.Layer

In [8]:
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
        
    def get_config(self):
        return {"factor": self.factor} 

In [ ]:
Note that you must implement the call() method for losses, layers (including
activation functions), and models, or the
call
__
__() method for regularizers,
initializers, and constraints. For metrics, things are a bit different, as you will
see now

Para pérdidas, capaz y modelos se usa call(). Para regularizadores, intializadores o constraints __call__()

### Métricas
Una métrica difiere de la pérdida en cuanto la función pérdida necesita ser derivable y distinta de cero en el dominio correspondiente, además de que no es necesario que sea fácilmente interpretable con quien la lee. Las métricas por el contrario deben tender a ser fáciles de interpretar

Definir una métrica personalizada es bastante parecido a definir una función pérdida, para ello utilizaré la misma función huber loss, usandóla como métrica

Para cada lote de entrenamiento Keras computa la función y lleva rastro de su promedio a lo largo de la época. A veces no se requiere que lleve el promedio, como es el caso de la precisión de clasificación binaria, porque es el cociente entre las predicciones acertadas de una clase y la suma entre las predicciones erradas y las acertadas, por lo tanto, acumular el promedio no refleja el valor de la métrica real. Estas mediciones que se actualizan de batch a batch se llaman streaming metrics. Lo que se hace es una clase que conozca los valores de predicciónes positivas y negativas y actualice esta métrica como tf.keras.metric.Precision. En cualquier punto de la ejecución se puede llamar al método result() para obtener el valor de la métrica o ver sus variables con el atributo variables y se pueden resetear con el método reset_states()

In [3]:
import tensorflow as tf
tf.random.set_seed(42)

@tf.function
def huber_fn(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    abs_error = tf.abs(error)
    quadratic = 0.5 * tf.square(error)
    linear = delta * (abs_error - 0.5 * delta)
    return tf.where(abs_error <= delta, quadratic, linear)

class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

- El constructor usa add_weight para crear las variables necesarias en cero. Se podrían usar variables normales, pero keras mantiene tarzabilidad de los tf.variables o más generalmente de los trackable
- update_state, calcula la función huber_fn, agrega el valor a la varaible total y le suma la cantidad de las predicciones a la variable count.
- result() calcula la métrica objetivo sobre todas las instancias. Keras llama siempre a update_state primero y después a result
- get_config está para que se guarde el threshold si se guarda el modelo

Cuando se define una métrica usando una función Keras automáticamente la llama para cada lote, y lleva rastro de la métrica para cada época. El único beneficio de esta clase es para guardar parámetros al guardar el modelo y que algunas métricas como la precisión no se realizan simplemente promediando entre batches.